# PDF 요약 챗봇 만들기
- 사용자가 PDF 파일을 업로드하면, 챗봇이 내용을 요약해주고 질문에도 답해주는 것

##  사용 도구
- LangChain (문서 QA 및 요약)
- PyPDFLoader (PDF 읽기)
- Chroma (벡터 저장소로 RAG 수행)
- OpenAI (LLM)

In [2]:
!pip install langchain langchain-openai chromadb unstructured pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 3.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.5 MB/s eta 0:00:0000:01
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=9f9973e27cc61360b9770f34c62c47d59641470f8a5eb471767677c073f9a7af
  Stored in directory: /Users/hojunhwang/Library/Caches/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: pydantic

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, load_summarize_chain
import os

os.environ["OPENAI_API_KEY"] = ""

# ✅ 1. Load PDF
loader = PyPDFLoader("Reglements_Noca.pdf")  # ← 요약하고 싶은 PDF 경로
docs = loader.load()

# ✅ 2. Split
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = splitter.split_documents(docs)

# ✅ 3. Embedding + Vector DB
embedding = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(split_docs, embedding)

# ✅ 4. Retrieval QA 챗봇
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

# ✅ 5. 질문 예시
print("Q: What is this PDF about?")
print(qa.run("What is this PDF about?"))

# ✅ 6. 전체 요약
summary_chain = load_summarize_chain(llm, chain_type="stuff")
print("\n📝 Summary of PDF:")
print(summary_chain.run(split_docs[:10]))  # 너무 많으면 일부만 사용


Q: What is this PDF about?


/var/folders/k0/_9hlrzgn55gcf4bpt7v94kz40000gn/T/ipykernel_89731/2443809555.py:28: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(qa.run("What is this PDF about?"))


This PDF contains information about the regulations and rules related to common areas, animals, landscaping, meeting agendas, convocation notices, proxies, and other aspects of a condominium or shared property. It outlines rules regarding animals in common areas, landscaping, meeting agendas, convocation notices, proxies for meetings, and the responsibilities of owners, tenants, and occupants in shared spaces.

📝 Summary of PDF:
The document outlines various regulations related to the use and maintenance of private and common areas in a condominium building. It covers rules regarding the design and use of private spaces, signage on the building facade, responsibilities of owners for damages, and regulations for parking and storing bicycles in common areas. The regulations aim to ensure the proper upkeep and respectful use of the property by all residents.


## 💡 응용 아이디어
- Streamlit 또는 Gradio로 인터페이스 추가
- PDF -> 요약 저장 -> 질문 응답 플로우 자동화
- 업로드한 PDF를 여러 개 비교 요약하기

## 패키지 설명
| 패키지 이름                 | 설명                                                                                                                                           |
| ---------------------- | -------------------------------------------------------------------------------------------------------------------------------------------- |
| **`langchain`**        | LLM 애플리케이션을 빠르게 만들 수 있는 파이썬 프레임워크입니다. 다양한 체인, 에이전트, 툴, 문서 처리 로직 등을 제공합니다. <br>→ `DocumentLoader`, `RetrievalQA`, `summarize_chain` 등을 제공합니다. |
| **`langchain-openai`** | LangChain에서 OpenAI의 LLM(GPT-3.5, GPT-4 등)을 쉽게 연동할 수 있게 해주는 모듈입니다. <br>→ 예: `ChatOpenAI`, `OpenAIEmbeddings`                                  |
| **`chromadb`**         | 오픈소스 벡터 데이터베이스입니다. 문서 임베딩을 저장하고, 쿼리(검색어)와 유사한 문서를 빠르게 찾아줍니다. <br>→ `Chroma`는 LangChain에서 가장 많이 쓰이는 기본 벡터 저장소입니다.                             |
| **`unstructured`**     | 다양한 문서 타입(PDF, HTML, Word, 이메일 등)을 구조화된 텍스트로 파싱해주는 라이브러리입니다. <br>→ LangChain 문서 로더들이 내부적으로 사용합니다.                                            |
| **`pypdf`**            | PDF 파일을 읽고 파싱하기 위한 파이썬 라이브러리입니다. <br>→ `PyPDFLoader`는 이걸 활용하여 PDF 문서를 페이지별로 로드합니다.                                                           |

